In [28]:
import kagglehub
alphiree_cardiovascular_diseases_risk_prediction_dataset_path = kagglehub.dataset_download('alphiree/cardiovascular-diseases-risk-prediction-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'cardiovascular-diseases-risk-prediction-dataset' dataset.
Data source import complete.


In [29]:
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go
import matplotlib.pyplot as plt
colors = px.colors.sequential.Plasma_r

# Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc

# Warnings
import warnings
warnings. filterwarnings('ignore')


In [30]:
import os
heart = pd.read_csv(os.path.join(alphiree_cardiovascular_diseases_risk_prediction_dataset_path, 'CVD_cleaned.csv'))

In [31]:
heart.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


In [32]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

In [33]:
heart.shape

(308854, 19)

In [34]:
# Checking if there are any null values in the dataset or not
heart.isnull().sum()

,0
General_Health,0
Checkup,0
Exercise,0
Heart_Disease,0
Skin_Cancer,0
Other_Cancer,0
Depression,0
Diabetes,0
Arthritis,0
Sex,0


In [35]:
# Convering the column names into lower case and replacing the space with an underscore
heart.columns = heart.columns.str.lower().str.replace(" ", "_")

#Changing the name of a big column

heart.rename(columns = {'height_(cm)' : 'height', 'weight_(kg)' : 'weight', 'green_vegetables_consumption' : 'vegetables_consumption', 'friedpotato_consumption' : 'potato_consumption'}, inplace = True)

In [36]:
# With the help of for loop, we will now check if there are any typos in the categorical columns or not
for col in heart.select_dtypes(include = "object"):
    print(f"Column name: {col}")
    print(heart[col].unique())
    print('\n', '-'*80, '\n')

Column name: general_health
['Poor' 'Very Good' 'Good' 'Fair' 'Excellent']

 -------------------------------------------------------------------------------- 

Column name: checkup
['Within the past 2 years' 'Within the past year' '5 or more years ago'
 'Within the past 5 years' 'Never']

 -------------------------------------------------------------------------------- 

Column name: exercise
['No' 'Yes']

 -------------------------------------------------------------------------------- 

Column name: heart_disease
['No' 'Yes']

 -------------------------------------------------------------------------------- 

Column name: skin_cancer
['No' 'Yes']

 -------------------------------------------------------------------------------- 

Column name: other_cancer
['No' 'Yes']

 -------------------------------------------------------------------------------- 

Column name: depression
['No' 'Yes']

 -------------------------------------------------------------------------------- 

Column name:

In [37]:
heart['checkup'] = heart['checkup'].replace('Within the past 2 years', 'Past 2 years')
heart['checkup'] = heart['checkup'].replace('Within the past year', 'Past 1 year')
heart['checkup'] = heart['checkup'].replace('Within the past 5 years', 'Past 5 years')
heart['checkup'] = heart['checkup'].replace('5 or more years ago', 'More than 5 years')


heart['diabetes'] = heart['diabetes'].replace('No, pre-diabetes or borderline diabetes', 'No Pre Diabetes')
heart['diabetes'] = heart['diabetes'].replace('Yes, but female told only during pregnancy', 'Only during pregnancy')

heart['age_category'] = heart['age_category'].replace('18-24', 'Young')
heart['age_category'] = heart['age_category'].replace('25-29', 'Adult')
heart['age_category'] = heart['age_category'].replace('30-34', 'Adult')
heart['age_category'] = heart['age_category'].replace('35-39', 'Adult')
heart['age_category'] = heart['age_category'].replace('40-44', 'Mid-Aged')
heart['age_category'] = heart['age_category'].replace('45-49', 'Mid-Aged')
heart['age_category'] = heart['age_category'].replace('50-54', 'Mid-Aged')
heart['age_category'] = heart['age_category'].replace('55-59', 'Senior-Adult')
heart['age_category'] = heart['age_category'].replace('60-64', 'Senior-Adult')
heart['age_category'] = heart['age_category'].replace('65-69', 'Elderly')
heart['age_category'] = heart['age_category'].replace('70-74', 'Elderly')
heart['age_category'] = heart['age_category'].replace('75-79', 'Elderly')
heart['age_category'] = heart['age_category'].replace('80+', 'Elderly')

In [38]:
col = ['alcohol_consumption', 'fruit_consumption', 'vegetables_consumption', 'potato_consumption']

for i in col:
    heart[i] = heart[i].astype(int)

<div style="border-radius:10px; border:#000000 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=black>📊 Insights:</font></h3>

* The are **0 null values** in the dataset
* Some **column names and their values** were changed
* **Data types** of a few columns were changed

In [39]:
# Define BMI ranges and labels for each group
bmi_bins = [12.02, 18.3, 26.85, 31.58, 37.8, 100]
bmi_labels = ['Underweight', 'Normal weight', 'Overweight', 'Obese I', 'Obese II']
heart['bmi_group'] = pd.cut(heart['bmi'], bins=bmi_bins, labels=bmi_labels, right=False)

In [40]:
column_to_move = heart.pop('bmi_group')
heart.insert(14, 'bmi_group', column_to_move)

In [41]:
heart['bmi_group'] = heart['bmi_group'].astype('object')

<div style="border-radius:10px; border:#000000 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=black>📊 Insights:</font></h3>

* New feature of **bmi_group** was created that consisted of bmi bins
* The **position** was changed and it's **data type** as well

In [42]:
heart.describe(include = 'O')

,general_health,checkup,exercise,heart_disease,skin_cancer,other_cancer,depression,diabetes,arthritis,sex,age_category,bmi_group,smoking_history
count,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854,308854
unique,5,5,2,2,2,2,2,4,2,2,5,5,2
top,Very Good,Past 1 year,Yes,No,No,No,No,No,No,Female,Elderly,Normal weight,No
freq,110395,239371,239381,283883,278860,278976,246953,259141,207783,160196,107513,136006,183590


In [43]:
heart.describe().T

,count,mean,std,min,25%,50%,75%,max
height,308854.0,170.615249,10.658026,91.00,163.00,170.00,178.00,241.00
weight,308854.0,83.588655,21.343210,24.95,68.04,81.65,95.25,293.02
bmi,308854.0,28.626211,6.522323,12.02,24.21,27.44,31.85,99.33
alcohol_consumption,308854.0,5.096366,8.199763,0.00,0.00,1.00,6.00,30.00
fruit_consumption,308854.0,29.835200,24.875735,0.00,12.00,30.00,30.00,120.00
vegetables_consumption,308854.0,15.110441,14.926238,0.00,4.00,12.00,20.00,128.00
potato_consumption,308854.0,6.296616,8.582954,0.00,2.00,4.00,8.00,128.00


<div style="border-radius:10px; border:#000000 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=black>📊 Insights:</font></h3>

* The risk factor analysis revealed that people who have a **poor health have more chances of heart disease**.
    
* In terms of **gender**, the data shows a **high ratio of male diagnosed with heart disease** and they have a **bit hight bmi as compared to females**
    
* According to age groups, the **elderly poeple have the highest count** for heart disease. However, **mid aged people having heart disease tend to have a higher bmi**
    
* In terms of **exercise**, it doesn't play a significant role. However, **smoking is an important factor for heart disease poeple**.
    
* The **alcohol consumption** has a significant impact on heart disease, however, it doesn't affect much when a person is in young age. Furthermore, the analysis revaled that, **fruit consumption isn't having much effect on poeple with heart disease**.
    
    
* Similarly, **green vegetables consumption** has a significant impact on heart disease, specially, it affects more when a person is in adult age. Furthermore, the analysis revaled that, **fried potato consumption isn't having any effect on poeple with heart disease**.

<div style="border-radius:10px; border:#000000 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=black>📊 Insights:</font></h3>

* The correlation analysis revealed that people who have a **depression have more chances of heart disease**.
    
* Similarly, **diabetes has a significant impact** on heart disease, specially, people with diabetes and heart disease **have a higher bmi on average**.

---
### 1. One-Hot Encoding
---

In [49]:
heart['heart_disease'] = heart['heart_disease'].map({'Yes':1, 'No':0})
cat=['sex', 'smoking_history']

OH_Encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH = OH_Encoder.fit_transform(heart[cat])
cols = OH_Encoder.get_feature_names_out(cat)
OH = pd.DataFrame(OH, columns=cols)
heart = heart.drop(cat,axis=1)
heart = pd.concat([heart, OH], axis =1)

---
### 2. Label Encoding
---

In [50]:
categorical_columns = ['general_health', 'checkup', 'exercise', 'skin_cancer', 'other_cancer', 'depression', 'diabetes', 'arthritis', 'age_category', 'bmi_group']

# Initialize LabelEncoder

label_encoder = LabelEncoder()

# Apply label encoding to each ordinal categorical column

for col in categorical_columns:
    heart[col] = label_encoder.fit_transform(heart[col])

---
### 3. Class Imbalance
---

In [51]:
# Checking the class Imbalance

heart['heart_disease'].value_counts()

,count
heart_disease,
0,283883
1,24971


In [52]:
X = heart.drop("heart_disease", axis = 1)
y = heart['heart_disease']

In [53]:
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

---
### 4. Splitting into training and testing
---

In [54]:
# Splitting the data into training and testing sets for diabetes balanced

X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

---
### 5. Feature Scaling
---

In [55]:
# Feature Scaling on diabetes dataset

scaler_d = StandardScaler()
X_train_scaled = scaler_d.fit_transform(X_train)
X_test_scaled = scaler_d.transform(X_test)

In [56]:
# Create the models

lr_d = LogisticRegression()
rf_d = RandomForestClassifier()

# Fit the models

lr_d.fit(X_train_scaled, y_train)
rf_d.fit(X_train_scaled, y_train)

# Make predictions

lr_pred_d = lr_d.predict(X_test_scaled)
rf_pred_d = rf_d.predict(X_test_scaled)

In [57]:
import joblib
import zipfile
import os

print("Starting model saving process...")

# 1. Define file names
model_filename = 'HeartRisk_model.joblib'
scaler_filename = 'HeartRisk_scaler.joblib'
zip_filename = 'HeartRisk_model_package.zip'

try:
    # 2. Save the ensemble model
    joblib.dump(rf_d, model_filename)
    print(f"Ensemble model saved successfully as: {model_filename}")

    # 3. Save the scaler (CRITICAL for deployment)
    joblib.dump(scaler_d, scaler_filename)
    print(f"Scaler saved successfully as: {scaler_filename}")

    # 4. Create a single zip file containing both assets
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(model_filename)
        zf.write(scaler_filename)

    print(f"All assets successfully zipped into: {zip_filename}")

    # 5. (Optional) Clean up the individual files after zipping
    os.remove(model_filename)
    os.remove(scaler_filename)
    print(f"Cleanup complete. Ready for deployment with {zip_filename}")

except Exception as e:
    print(f"An error occurred during saving/zipping: {e}")

Starting model saving process...
Ensemble model saved successfully as: HeartRisk_model.joblib
Scaler saved successfully as: HeartRisk_scaler.joblib
All assets successfully zipped into: HeartRisk_model_package.zip
Cleanup complete. Ready for deployment with HeartRisk_model_package.zip


In [58]:
def plot_classification_report(report, title):
    lines = report.split('\n')[2:-5]
    classes = []
    precision = []
    recall = []
    f1_score = []
    support = []
    for line in lines:
        row_data = line.split()
        classes.append(row_data[0])
        precision.append(float(row_data[1]))
        recall.append(float(row_data[2]))
        f1_score.append(float(row_data[3]))
        support.append(int(row_data[4]))

    fig = go.Figure()
    fig.add_trace(go.Bar(x=classes, y=precision, name='Precision', marker_color = colors[0]))
    fig.add_trace(go.Bar(x=classes, y=recall, name='Recall', marker_color = colors[1]))
    fig.add_trace(go.Bar(x=classes, y=f1_score, name='F1-Score', marker_color = colors[2]))

    fig.update_layout(title=title, xaxis_title='Class', yaxis_title='Score', barmode='group', xaxis={'categoryorder': 'total descending'}, plot_bgcolor='white')

    fig.show()

In [59]:
# Classification reports for different algorithms

lr_d_report = classification_report(y_test, lr_pred_d)

rf_d_report = classification_report(y_test, rf_pred_d)


In [60]:
# Plot classification reports

print("="*40, "Logistic regression report:", "="*45, '\n')
print(lr_d_report)
plot_classification_report(lr_d_report, "Logistic Regression Classification Report Visualization")


print("="*40, "Random forest report:", "="*45, '\n')
print(rf_d_report)
plot_classification_report(rf_d_report, "Random Forest Classification Report")

======================================== Logistic regression report: ============================================= 

              precision    recall  f1-score   support

           0       0.72      0.72      0.72     85071
           1       0.72      0.72      0.72     85259

    accuracy                           0.72    170330
   macro avg       0.72      0.72      0.72    170330
weighted avg       0.72      0.72      0.72    170330



======================================== Random forest report: ============================================= 

              precision    recall  f1-score   support

           0       0.92      0.95      0.94     85071
           1       0.95      0.92      0.93     85259

    accuracy                           0.93    170330
   macro avg       0.93      0.93      0.93    170330
weighted avg       0.93      0.93      0.93    170330



<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=black>Conclusion:</font></h3>
    
---    
In conclusion, this notebook has provided valuable insights into the risk factors and predictive measures for heart diseases. Through meticulous data analysis, feature engineering, and advanced machine learning techniques, we have uncovered significant associations between various factors and disease occurrences.

 ---   
**Heart Disease Insights:**
         
    
In conclusion, the analysis of the dataset yielded several valuable insights. Null values were effectively handled, and data preprocessing included changes in column names, values, and data types. A new feature, 'bmi_group,' was created based on BMI bins and underwent data type and position adjustments. The overall health profile of the dataset indicated that most individuals were in good health, with normal weight and recent checkups, along with regular exercise. Furthermore, key factors contributing to heart disease were identified, including poor health, higher BMI, gender, age group, smoking, alcohol consumption, depression, and diabetes. Additionally, machine learning analysis using Random Forest demonstrated the best performance with an impressive F1-score and AUC of 0.93, suggesting its effectiveness in predicting heart disease in this context.

---    
    
**What's next?**
    
These discoveries emphasize the significance of personalized interventions and risk evaluation within public health strategies. By integrating these observations into clinical procedures, we can elevate disease prevention and treatment, ultimately leading to enhanced health outcomes on a broader scale.
    
---

***Thankyou so much for veiwing the notebook. If you liked the notebook, please upvote and give a feedback to improve it further ❤️***